# IBM_Data Science Capstone Project

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
# Get the link
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data1 = requests.get(url).text

In [3]:
#data1

In [4]:
# Scrape data from that link using BeautifulSoup
data = BeautifulSoup(data1, 'xml')

In [5]:
data_new = data.find('table')
#data_new

In [6]:
# Create an empty dataframe with three columns:
columns = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns=columns)
df

,PostalCode,Borough,Neighborhood


In [7]:
# get data for PostalCode, Borough, Neighborhood
for tr in data_new.find_all('tr'):
    empty_df=[]
    for td in tr.find_all('td'):
        empty_df.append(td.text.strip())
    if len(empty_df) ==3:
        df.loc[len(df)] = empty_df

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
# find all Borough that are "Not assigned" and remove them
df = df[df['Borough'] !='Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
# groupby Postalcode
df_group = df.groupby(["PostalCode","Borough"])['Neighborhood'].apply(lambda x: ','.join(x))
df_group = df_group.reset_index(drop=False)
df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df.shape

(103, 3)

## Part 2

In [12]:
# get latitude and longitude from csv file
geo_data = pd.read_csv("E:\TANHTRAN\Education\Coursera\IBM Data Science\Course9_Capstone\Week 3\Geospatial_Coordinates.csv")

In [13]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# merge geo_data with df based on column "PostalCode"
# But we need to change the name of the column "Postal Code" of geo_data to "PostalCode" to match with the name in df in order to merge
geo_data.rename(columns={'Postal Code':"PostalCode"}, inplace=True) # inplace = True to replace that for later use
geo_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
# Check again data info of df:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
# Now, lets merge geo_data with df, and named as df_new
df_new = pd.merge(geo_data, df, on = "PostalCode") # merge 2 dataframes based on column "PostalCode"
df_new.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [17]:
# We can change the order of the columns
df_new = df_new[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

In [18]:

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [24]:
toronto_data = df_new[df_new['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [37]:
# Get geographical coordinates of Toronto
address = "Toronto"
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinate of Toronto are {}, {}.".format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto

In [42]:
# Create a map of Toronto with neighborhood superimposed on top
map_toronto = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to the map above
for lat, lng, borough, neighborhood in zip (toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill='True',
        fill_color='yellow',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

### Foursquare API

In [50]:
# Now, utilize the Foursquare API to explore the neighborhood and segment them
# Define Foursqaure Credentials and Version
CLIENT_ID = 'CGNRCJD3RMHOEC1ZZEVECOQ3OAOJWPY52SRXOHKUOA4T4DWD'
CLIENT_SECRET = 'OLRFWTF3W3EH15ADBULC3UISP3ZXK4SF3AOCM5IOR3DCVG0O'
VERSION = '20201201'
LIMIT = 100

print("Credentials:")
print("CLIENT_ID: " + CLIENT_ID)
print("CLIENT_SECRET: " + CLIENT_SECRET)


Credentials:
CLIENT_ID: CGNRCJD3RMHOEC1ZZEVECOQ3OAOJWPY52SRXOHKUOA4T4DWD
CLIENT_SECRET: OLRFWTF3W3EH15ADBULC3UISP3ZXK4SF3AOCM5IOR3DCVG0O


In [51]:
# Explore first neighborhood in the toronto_data
toronto_data.loc[0,'Neighborhood']

'The Beaches'

In [52]:
# get the latitude and longitude of the first neighborhood
neighborhood_latitude = toronto_data.loc[0,'Latitude']
neighborhood_longitude = toronto_data.loc[0,'Longitude']
neighborhood_name = toronto_data.loc[0,'Neighborhood']

print("Latitude and Longitude values of {} are {}, {}".format(neighborhood_name,neighborhood_latitude, neighborhood_longitude))

Latitude and Longitude values of The Beaches are 43.67635739999999, -79.2930312


In [68]:
### Now, lets get the top 100 venues that are in "The Beaches" within a radius of 1000 meters
# First, create the GET request URL
Limit = 100
Radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,CLIENT_SECRET,VERSION, neighborhood_latitude, neighborhood_longitude, Radius, Limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CGNRCJD3RMHOEC1ZZEVECOQ3OAOJWPY52SRXOHKUOA4T4DWD&client_secret=OLRFWTF3W3EH15ADBULC3UISP3ZXK4SF3AOCM5IOR3DCVG0O&v=20201201&ll=43.67635739999999,-79.2930312&radius=1000&limit=100'

In [69]:
# next, send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd5a52c6ba8132ab79e8391'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 75,
  'suggestedBounds': {'ne': {'lat': 43.685357409000005,
    'lng': -79.28061062898105},
   'sw': {'lat': 43.66735739099998, 'lng': -79.30545177101895}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'lab

In [70]:
# define a function called get_category_type to extract the category of the venue
def get_catogery_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [71]:
# Now, clean the json and structure it into a pandas dataframe
from pandas.io.json import json_normalize # first import json_normalize

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]

# apply the function get_category_type
nearby_venues['venue.categories'] = nearby_venues.apply(get_catogery_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

75 venues were returned by Foursquare.


In [73]:
nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
2,The Beech Tree,Gastropub,43.680493,-79.288846
3,Beaches Bake Shop,Bakery,43.680363,-79.289692
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993


### Now, lets write a function named: getNearbyVenues and apply to explore neighborhood in Toronto

In [107]:
### Now, lets write a function named: getNearbyVenues and apply to explore neighborhood in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius = 1000):
    LIMIT=100
    VERSION='20201201'
    venues_list =[]
    for name, lat, lng in zip (names, latitudes, longitudes):
        print(name)
        
        # create the API request url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, lat, lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name'])
            for v in results ])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                                 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category' ]
    return(nearby_venues)

In [108]:
# run the above function for each neighborhood and called "toronto_venues"
toronto_venues = getNearbyVenues(names = toronto_data['Neighborhood'], 
                                 latitudes = toronto_data['Latitude'], longitudes = toronto_data['Longitude'])


The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [102]:
toronto_venues.shape

(3176, 7)

In [120]:
toronto_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3171,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Measurement Park,43.666916,-79.312631,Park
3172,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Breakfast Club,43.662811,-79.310174,Breakfast Spot
3173,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Carters Landing,43.662414,-79.309898,Bistro
3174,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Country Style,43.667662,-79.312006,Coffee Shop
3175,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Ashbridges Bay Yacht Club (ABYC),43.658210,-79.311004,Harbor / Marina


In [110]:
# check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",50,50,50,50,50,50
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


In [112]:
# count how many unique categories can be curated from toronto_venues
print("There are {} uniques categories.".format(len(toronto_venues['Venue Category'].unique())))


There are 280 uniques categories.


### Analyze each neighborhood

In [121]:
# onehot coding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first place
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
# check the size of toronto_onehot
toronto_onehot.shape

(3176, 280)

In [123]:
# group rows by "neighborhood" and take the mean
toronto_group = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_group

,Neighborhood,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,Berczy Park,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.010000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.01,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.00,0.071429,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
4,Central Bay Street,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.01,0.020000,0.000000,0.00,0.000000,0.00,0.010000,0.020000
5,Christie,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.020000,0.000000,0.00,0.010000,0.00,0.010000,0.000000
6,Church and Wellesley,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.000000,0.000000,0.00,0.010000,0.00,0.000000,0.020000
7,"Commerce Court, Victoria Hotel",0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
8,Davisville,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.010000,0.010000,0.00,0.010000,0.00,0.010000,0.010000
9,Davisville North,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.010000,0.010000,0.00,0.010000,0.00,0.010000,0.020000


In [124]:
# check the size of toronto_group
toronto_group.shape

(39, 280)

In [125]:
# Print each neighborhood along with the top 5 most common venues
num_top_venues=5
for neighbor in toronto_group['Neighborhood']:
    print("neighbor")
    temp = toronto_group[toronto_group['Neighborhood']==neighbor].T.reset_index()
    temp.columns = ['Venue', 'Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency':2})
    print(temp.sort_values("Frequency", ascending=False).reset_index(drop=True).head(num_top_venues))
    
    

neighbor
                 Venue  Frequency
0          Coffee Shop       0.10
1                 Café       0.07
2                Hotel       0.05
3  Japanese Restaurant       0.05
4             Beer Bar       0.04
neighbor
                    Venue  Frequency
0             Coffee Shop       0.07
1                    Café       0.07
2                     Bar       0.06
3              Restaurant       0.04
4  Furniture / Home Store       0.04
neighbor
              Venue  Frequency
0              Park       0.10
1       Coffee Shop       0.08
2           Brewery       0.06
3       Pizza Place       0.06
4  Sushi Restaurant       0.04
neighbor
              Venue  Frequency
0       Coffee Shop       0.14
1   Harbor / Marina       0.14
2              Café       0.14
3  Sculpture Garden       0.07
4           Dog Run       0.07
neighbor
                 Venue  Frequency
0          Coffee Shop       0.09
1  Japanese Restaurant       0.03
2                 Café       0.03
3     Sushi Restauran

##### Lets put the above outputs into a dataframe


In [126]:
# define a function sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [130]:
# create a dataframe and show top 10 venues for each neighborhood
num_top_venues = 10
indicators = ['st','nd', 'rd']

# create columns based on number of top venues
columns = ['Neighborhood']

# import numpy
import numpy as np

for ind in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most common venues".format(ind+1, indicators[ind]))
    except: 
        columns.append('{}th Most common venues'.format(ind+1))
# create a new dataframe
neighborhoods_venue_sorted = pd.DataFrame(columns=columns)
neighborhoods_venue_sorted['Neighborhood'] = toronto_group['Neighborhood']

for ind in np.arange(toronto_group.shape[0]):
    neighborhoods_venue_sorted.iloc[ind,1:] = return_most_common_venues(toronto_group.iloc[ind,:], num_top_venues)

In [131]:
neighborhoods_venue_sorted.head()

,Neighborhood,1st Most common venues,2nd Most common venues,3rd Most common venues,4th Most common venues,5th Most common venues,6th Most common venues,7th Most common venues,8th Most common venues,9th Most common venues,10th Most common venues
0,Berczy Park,Coffee Shop,Café,Japanese Restaurant,Hotel,Park,Beer Bar,Restaurant,Gastropub,Farmers Market,Grocery Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Arts & Crafts Store,Supermarket
2,"Business reply mail Processing Centre, South C...",Park,Coffee Shop,Pizza Place,Brewery,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Bakery,Snack Place,Juice Bar
3,"CN Tower, King and Spadina, Railway Lands, Har...",Harbor / Marina,Café,Coffee Shop,Scenic Lookout,Sculpture Garden,Airport,Dog Run,Dance Studio,Park,Garden
4,Central Bay Street,Coffee Shop,Ramen Restaurant,Park,Café,Japanese Restaurant,Sushi Restaurant,Gastropub,Plaza,Mexican Restaurant,Juice Bar


### Cluster neighborhood

In [133]:
# Run k-means to cluster the neighborhood into 5 clusters
kclusters = 5

toronto_group_cluster = toronto_group.drop('Neighborhood', axis=1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_group_cluster)

kmeans.labels_[0:10]

array([2, 2, 3, 4, 0, 0, 0, 2, 3, 3])

In [134]:
# create a new dataframe that include cluster and top 10 most common venues
neighborhoods_venue_sorted.insert(0,'Cluster',kmeans.labels_)
toronto_merged = toronto_data

# merge toronto_group with toronto_data to get latitude and longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venue_sorted.set_index('Neighborhood'), on ="Neighborhood")

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most common venues,2nd Most common venues,3rd Most common venues,4th Most common venues,5th Most common venues,6th Most common venues,7th Most common venues,8th Most common venues,9th Most common venues,10th Most common venues
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Japanese Restaurant,Beach,Grocery Store,Bakery,Sandwich Place,Caribbean Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Café,Pub,Italian Restaurant,Bank,Discount Store,Spa,Restaurant,Ramen Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Indian Restaurant,Coffee Shop,Grocery Store,Park,Restaurant,Beach,Brewery,Bakery,Butcher,Café
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Coffee Shop,Brewery,American Restaurant,Bar,Vietnamese Restaurant,Bakery,Diner,French Restaurant,Sushi Restaurant,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bookstore,Coffee Shop,College Gym,Trail,College Quad,Café,Gym / Fitness Center,Falafel Restaurant,Event Space


In [140]:
### Finally, visualize the clusters
# first, create a map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start=10)

# set color
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup=label,
        color = rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity= 0.5).add_to(map_clusters)
map_clusters